### (Hands-on 9장) Up and running with TensorFlow  (7/11)

### 텐서뽀개기, 이상열

- 스터디 : 캐글뽀개기 - 텐서뽀개기
- 페이스북 : <https://www.facebook.com/groups/kagglebreak/>
- github : <https://github.com/KaggleBreak/tensorbreak>
- 구글드라이브 : 
<https://drive.google.com/drive/folders/0B2l0iH28o85xM3A3TVhGdkFHb3M>


### Intro
- TensorFlow는 수치 계산을 위한 강력한 오픈 소스 소프트웨어 라이브러리이며, 특히 대규모 기계 학습을 위해 매우 적합하고 미세 조정됩니다. 기본 원칙은 간단합니다. 먼저 파이썬에서 수행 할 계산 그래프 (예 : 그림 9-1의 그래프)를 정의한 다음 TensorFlow는 그래프를 가져 와서 최적화 된 C ++ 코드를 사용하여 효율적으로 실행합니다.

<img src ='./img/chap9_1.png'>

- 가장 중요한 점은 그래프를 여러 개의 청크로 분할하고 여러 개의 CPU 또는 GPU에서 병렬로 실행할 수 있다는 것입니다 (그림 9-2 참조). 

- TensorFlow는 분산 컴퓨팅을 지원하기 때문에 계산을 수백 대의 서버로 나누어 엄청난 양의 엄청난 양의 훈련을 엄청나게 훈련시킬 수 있습니다 (12 장 참조). TensorFlow는 수십억 개의 인스턴스와 수백만 개의 기능으로 구성된 교육 세트에서 수백만 개의 매개 변수가있는 네트워크를 교육 할 수 있습니다. TensorFlow가 존재했기 때문에 이것은 놀랄 일이 아닙니다.

<img src ='./img/chap9_2.png'>

- TensorFlow가 2015 년 11 월에 오픈 소스가되었을 때 이미 Deep Learning을위한 많은 오픈 소스 라이브러리가 있었고 (표 9-1은 몇 가지를 나열했습니다), TF의 기능 중 상당 부분은 이미 하나의 라이브러리 또는 다른 라이브러리에 존재했습니다. 그럼에도 불구하고 TensorFlow의 깔끔한 설계, 확장성, 유연성 및 훌륭한 문서 (Google의 이름은 말할 것도 없음)가 신속하게 목록 상단으로 올라갔습니다. TF의 하이라이트 중 일부는 다음과 같습니다.

    - Linux 및 MacOSX 뿐만 아니라 iOS 및 Android를 포함한 모바일 장치에서도 실행됩니다.
    - Scikit-Learn와 호환되는 TF.Learn (tensorflow.con trib.learn)라는 매우 간단한 파이썬 API를 제공합니다. 사용자가 볼 수 있듯 이이를 사용하여 교육 할 수 있습니다
    - 또한 TF-slim (tensorflow.contrib.slim)이라는 간단한 API를 제공하여 신경망을 작성, 교육 및 평가하는 작업을 단순화합니다.
    - Keras 또는 Pretty Tensor와 같은 TensorFlow 상단에 몇 가지 다른 고급 API가 독립적으로 구축되었습니다.
    - 주요 파이썬 API는 생각할 수있는 신경망 아키텍처를 비롯하여 모든 종류의 계산을 생성 할 수있는 훨씬 더 많은 유연성 (높은 복잡성으로 인해)을 제공합니다.
    - 특히 ML 네트워크를 구축하는 데 필요한 많은 ML 작업에 대해 매우 효율적인 C++ 구현을 포함합니다. 또한 자신의 고성능 조작을 정의하는 C++ API가 있습니다.
    - 비용 기능을 최소화하는 매개 변수를 검색 할 수 있는 몇 가지 고급 최적화 노드를 제공합니다. TensorFlow는 사용자가 정의한 함수의 그라디언트를 자동으로 계산하기 때문에 사용하기가 매우 쉽습니다. 이를 자동 분산 (autodi)이라고합니다.
    - 또한 TensorBoard라는 훌륭한 시각화 도구가 있어 계산 그래프를 탐색하고 학습 곡선을 보는 등의 작업을 수행 할 수 있습니다.
    - Google은 TF 그래프 (https://cloud.google.com/ml/)를 실행하기 위해 클라우드 서비스도 시작했습니다.

<img src ='./img/chap9_3.png'>


### Installation (생략)

### Setup
- 먼저,이 노트가 파이썬 2와 3에서 잘 작동하는지 확인하고, 몇 가지 공통 모듈을 가져오고, Matplotlib 플롯이 인라인으로 표시되도록 하고 그림을 저장하는 함수를 준비하십시오.

In [213]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

### Creating your first graph and running it in a session

- 다음 코드는 그림 9-1에 표시된 그래프를 만듭니다.

In [214]:
import tensorflow as tf

reset_graph()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

f=x*x*y+y+2

In [215]:
x

<tf.Variable 'x:0' shape=() dtype=int32_ref>

In [216]:
y

<tf.Variable 'y:0' shape=() dtype=int32_ref>

In [217]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

- 이해해야 할 가장 중요한 것은 이 코드가 실제로는 (특히 마지막 줄) 보이더라도 실제로는 계산을 수행하지 않는다는 것입니다. 
- 단지 계산 그래프를 생성합니다. 실제로 변수조차도 아직 초기화되지 않았습니다. 
- 이 그래프를 평가하려면 TensorFlow 세션을 열고 이를 사용하여 변수를 초기화하고 f를 평가해야 합니다. 
- TensorFlow 세션은 CPU 및 GPU와 같은 장치에 작업을 배치하고 실행하며 모든 변수 값을 보유합니다. 다음 코드는 세션을 생성하고, 변수를 초기화하고, f를 평가 한 다음 세션을 닫음으로써 자원을 확보합니다.

In [93]:
sess = tf.Session()

In [94]:
sess.run(x.initializer)

In [95]:
sess.run(y.initializer)

In [96]:
result = sess.run(f)

In [98]:
print(result)

42


In [99]:
sess.close()

In [100]:
with tf.Session() as sess: 
    x.initializer.run() 
    y.initializer.run() 
    result = f.eval()

- with 블록의 세션은 기본 세션으로 설정됩니다. 
- x.initializer.run()을 호출하는 것은 tf.get_default_session().run (x.initializer)를 호출하는 것과 동일하며, 마찬가지로 f.eval()은 tf.get_default_session().run(f)를 호출하는 것과 동일합니다. 이렇게하면 코드를 읽기 쉽게 만듭니다. 또한 세션이 블록 끝에서 자동으로 닫힙니다.
- 모든 단일 변수에 대해 이니셜 라이저를 수동으로 실행하는 대신 initialize_all_variable() 함수를 사용할 수 있습니다. 실제로 초기화를 실제로 수행하지는 않으며 그래프가 실행될 때 모든 변수를 초기화 할 노드를 그래프로 만듭니다.

In [218]:
init = tf.global_variables_initializer()

In [219]:
with tf.Session() as sess:
    init.run() # actually initialize all the variables 
    result = f.eval()

- Jupyter 내부 또는 Python 셸 내에서 Interactive Sesion을 만드는 것이 더 좋습니다. 
- 일반 세션과의 유일한 차이점은 자동 세션이 생성되면 자동으로 기본 세션으로 설정되므로 with 블록이 필요하지 않습니다 (세션을 완료 할 때 세션을 수동으로 닫아야 함) :

In [220]:
sess = tf.InteractiveSession()

In [221]:
init.run()

In [222]:
result = f.eval()

In [223]:
print(result)

42


In [224]:
sess.close()

- TensorFlow 프로그램은 일반적으로 두 부분으로 나뉩니다. 
- 첫 번째 부분은 계산 그래프를 작성하고 두 번째 부분은 실행 단계입니다. 
- 구축 단계에서는 일반적으로 ML 모델과 이를 계발하는 데 필요한 계산을 나타내는 계산 그래프를 작성합니다. 
- 실행 단계는 일반적으로 반복적으로 교육 단계를 평가하는 루프를 실행합니다 (예 : 미니 배치 당 하나의 단계). 모델 매개 변수가 점진적으로 개선됩니다. 아래 예를 통해 살펴 보겠습니다.

### Managing graphs

- 생성한 노드는 자동으로 기본 그래프에 추가됩니다.

In [108]:
reset_graph()
x1 = tf.Variable(1)

In [109]:
x1.graph is tf.get_default_graph()

True

- 대부분의 경우 이 방법이 좋지만 때로는 여러 개의 독립적 인 그래프를 관리하고자 할 수 있습니다. 
- 새 그래프를 만들고 일시적으로 기본 그래프를 with 블록 안에 만들면 다음과 같이 할 수 있습니다.

In [228]:
graph = tf.Graph()

In [229]:
with graph.as_default():
    x2 = tf.Variable(2)

In [230]:
x2.graph is graph

True

In [113]:
x2.graph is tf.get_default_graph()

False

- Jupyter (또는 python 쉘)에서는 실험하는 동안 동일한 명령을 두 번 이상 실행하는 것이 일반적입니다. 
- 결과적으로 많은 중복 노드가 포함 된 기본 그래프로 끝날 수 있습니다. 한 가지 해결책은 Jupyter 커널 (또는 파이썬 쉘)을 다시 시작하는 것이지만 더 편리한 해결책은 tf.reset_default_graph()  -- reset_graph() 를 실행하여 기본 그래프를 재설정하는 것입니다.


### Lifecycle of a node value

- 노드를 평가할 때 TensorFlow는 의존하는 노드 집합을 자동으로 결정하고 이 노드를 먼저 평가합니다. 예를 들어, 다음 코드를 고려하십시오.

In [114]:
w = tf.constant(3) 
x=w+2 
y=x+5 
z=x*3
with tf.Session() as sess: 
    print(y.eval()) # 10 
    print(z.eval())

10
15


- 첫째,이 코드는 매우 간단한 그래프를 정의합니다. 
- 다음 세션을 시작하고 그래프를 실행하여 y를 평가합니다. TensorFlow는 y가 x에 의존하고 x는 w에 따라 달라지므로 자동으로 w, x, y를 평가하고 y 값을 반환합니다. 마지막으로 코드는 그래프를 실행하여 z를 평가합니다. 다시 한번, TensorFlow는 먼저 w와 x를 평가해야 함을 감지합니다. w와 x의 이전 평가 결과를 재사용하지 않는다는 점에 유의해야합니다. 간단히 말해 위의 코드는 w와 x를 두 번 평가합니다.
- 모든 노드 값은 그래프 실행 사이에 세션에 의해 유지되는 변수 값을 제외하고는 그래프 실행 사이에서 삭제됩니다 (큐와 판독기는 12 장에서 보듯이 일부 상태를 유지함). 변수는 초기화가 실행되면 수명이 시작되고 세션이 닫히면 끝납니다.
- 위 코드에서와 같이 w와 x를 두 번 평가하지 않고 y와 z를 효율적으로 계산하려면 다음 코드와 같이 TensorFlow에게 하나의 그래프 실행에서 y와 z를 모두 평가하도록 요청해야합니다.

In [115]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z]) 
    print(y_val) # 10 
    print(z_val) # 15

10
15


- 단일 프로세스 TensorFlow에서 여러 세션은 같은 그래프를 재사용하더라도 각 세션마다 상태를 공유하지 않습니다 (각 세션에는 모든 변수의 자체 복사본이 있음). Distributed TensorFlow (제 12 장 참조)에서는 가변 상태가 세션이 아닌 서버에 저장되므로 여러 세션이 동일한 변수를 공유 할 수 있습니다.

### Linear Regression with TensorFlow

- TensorFlow 작업은 임의의 수의 입력을 가져 와서 여러 출력을 생성 할 수 있습니다. 예를 들어, 가산 연산은 각각 2 개의 입력을 취하여 1 개의 출력을 생성합니다. 상수와 변수는 입력이 없습니다 (소스 작업이라고 함). 

- 인풋과 아웃풋은 tenors라고 불리는 다차원 배열입니다. NumPy 배열과 마찬가지로 텐서는 유형과 모양을 가집니다. 실제로 파이썬 API에서 텐서는 단순히 NumPy ndarrays로 표현됩니다. 대개는 부동 소수점을 포함하지만 문자열 (임의의 바이트 배열)을 전달하는데도 사용할 수 있습니다.

- 위의 예제에서, 텐서는 단지 하나의 스칼라 값을 포함하고 있지만 물론 어떤 모양의 배열에 대해서도 계산을 수행 할 수 있습니다. 예를 들어 다음 코드는 2 차원 배열을 조작하여 캘리포니아 주택 데이터 세트 (2 장에서 소개 함)에 선형 회귀를 수행합니다. 먼저 데이터 집합을 가져 와서 모든 학습 인스턴스에 추가 바이어스 입력 기능 (x0 = 1)을 추가합니다 (NumPy를 사용하여 수행되므로 즉시 실행 됨)

- 이 데이터와 대상을 보유하는 두 개의 TensorFlow 상수 노드 X와 Y를 만들고 Tensor-Flow에서 제공하는 일부 매트릭스 작업을 사용하여 theta를 정의합니다. 이 행렬 함수 (transpose (), matmul () 및 matrix_inverse ())는 자명하지만, 평소와 같이 계산을 즉시 수행하지 않고 대신 그래프를 실행할 때 그래프를 수행하는 노드를 만듭니다 . theta의 정의는 다음과 일치한다는 것을 알 수 있습니다.

- 정규 방정식 (θ = T · -1 · T ·, 4 장 참조). 마지막으로 코드는 세션을 만들고이를 사용하여 theta를 평가합니다.

In [116]:
from sklearn.datasets import fetch_california_housing
import numpy as np
reset_graph()

In [233]:
np.ones((m, 1))

array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]])

In [235]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [236]:
housing_data_plus_bias

array([[   1.        ,    8.3252    ,   41.        , ...,    2.55555556,
          37.88      , -122.23      ],
       [   1.        ,    8.3014    ,   21.        , ...,    2.10984183,
          37.86      , -122.22      ],
       [   1.        ,    7.2574    ,   52.        , ...,    2.80225989,
          37.85      , -122.24      ],
       ..., 
       [   1.        ,    1.7       ,   17.        , ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.        ,    1.8672    ,   18.        , ...,    2.12320917,
          39.43      , -121.32      ],
       [   1.        ,    2.3886    ,   16.        , ...,    2.61698113,
          39.37      , -121.24      ]])

In [232]:
housing

{'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n    http://lib.stat.cmu.edu/\n\nThe data contains 20,640 observations on 9 variables.\n\nThis dataset contains the average house value as target variable\nand the following input variables (features): average income,\nhousing average age, average rooms, average bedrooms, population,\naverage occupation, latitude, and longitude in that order.\n\nReferences\n----------\n\nPace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,\nStatistics and Probability Letters, 33 (1997) 291-297.\n\n',
 'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ..., 
        [   1.7       ,   17.        ,    

In [119]:
m, n

(20640, 8)

In [120]:
housing_data_plus_bias

array([[   1.        ,    8.3252    ,   41.        , ...,    2.55555556,
          37.88      , -122.23      ],
       [   1.        ,    8.3014    ,   21.        , ...,    2.10984183,
          37.86      , -122.22      ],
       [   1.        ,    7.2574    ,   52.        , ...,    2.80225989,
          37.85      , -122.24      ],
       ..., 
       [   1.        ,    1.7       ,   17.        , ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.        ,    1.8672    ,   18.        , ...,    2.12320917,
          39.43      , -121.32      ],
       [   1.        ,    2.3886    ,   16.        , ...,    2.61698113,
          39.37      , -121.24      ]])

In [237]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [238]:
X

<tf.Tensor 'X:0' shape=(20640, 9) dtype=float32>

In [239]:
y

<tf.Tensor 'y_1:0' shape=(20640, 1) dtype=float32>

In [240]:
XT

<tf.Tensor 'transpose:0' shape=(9, 20640) dtype=float32>

In [122]:
with tf.Session() as sess: 
    theta_value = theta.eval()

In [123]:
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

In [124]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654266e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


In [125]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


- 이 코드가 NumPy를 사용하여 직접적으로 수식을 계산하는 것보다 TensorFlow가 자동으로 GPU 카드에서 이 코드를 실행할 수 있다는 점이 이점입니다 (GPU를 지원하는 TensorFlow를 설치했다면 12 장 참조)

### Implementing Gradient Descent

- Normal Equations 대신 Batch Gradient Descent (4 장 참조)를 사용해 보겠습니다. 먼저 그라디언트를 수동으로 계산하여 이를 수행한 다음 TF가 자동으로 그라디언트를 계산할 수 있도록 TenorFlow의 자동 분산 기능을 사용하고 마지막으로 TessorFlow의 몇 가지 옵티마이저를 사용합니다.

### Manually computing the gradients
- 다음 코드는 몇 가지 새로운 요소를 제외하고는 상당히 자명합니다.
    - random_uniform() 함수는 그래프에 NumPy의 rand() 함수와 마찬가지로 모양과 값 범위가 주어진 임의의 값을 포함하는 텐서를 생성하는 노드를 만듭니다.
    - assign() 함수는 변수에 새 값을 할당 할 노드를 만듭니다. 이 경우 Batch Gradient Descent 단계 θ(next step) = θ-η∇θMSE θ를 구현합니다.
    - 메인 루프는 트레이닝 단계를 반복 실행하고 (n_epochs times) 100 회 반복 할 때마다 현재 Mean Squared Error (mse)를 인쇄합니다. 매 반복마다 MSE가 내려가는 것을 볼 수 있습니다.

In [241]:
n_epochs = 1000
learning_rate = 0.01

In [242]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [244]:
scaled_housing_data_plus_bias

array([[ 1.        ,  2.34476576,  0.98214266, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.        ,  1.7826994 ,  1.85618152, ..., -0.02584253,
         1.03850269, -1.33282653],
       ..., 
       [ 1.        , -1.14259331, -0.92485123, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [ 1.        , -1.05458292, -0.84539315, ..., -0.09122515,
         1.77823747, -0.87362627],
       [ 1.        , -0.78012947, -1.00430931, ..., -0.04368215,
         1.75014627, -0.83369581]])

In [128]:
reset_graph()
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.714501
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.555572
Epoch 400 MSE = 0.548812
Epoch 500 MSE = 0.543636
Epoch 600 MSE = 0.539629
Epoch 700 MSE = 0.536509
Epoch 800 MSE = 0.534068
Epoch 900 MSE = 0.532147


### using autodiff (미분)

- 위의 코드는 정상적으로 작동하지만 비용 함수(MSE)에서 그라디언트를 수학적으로 유도해야 합니다. 
- 선형 회귀 분석의 경우에는 비교적 쉽지만, deep neural networks를 사용하여 이 작업을 수행해야 한다면 두통이 심해질 수 있습니다. 지루하고 오류가 발생하기 쉽습니다. 
- 기호 편차를 사용하여 편미분 방정식의 방정식을 자동으로 찾을 수 있지만 결과 코드가 반드시 효율적이지는 않습니다.

In [129]:
def my_func(a, b): 
    z=0
    for i in range(100): 
        z=a*np.cos(z+i)+z*np.sin(a-i)
    return z

- 다행스럽게도 TF의 autodiff 기능은 자동으로 효율적으로 그라디언트를 계산할 수 있습니다.
- 이전 섹션의 Gradient Descent 코드의 gradients = ... 라인을 다음 라인으로 바꾸기 만하면 코드가 계속 잘 작동합니다.

- f(x) = exp(exp(exp(x)))
- f'(x) = exp(x)×exp(exp(x))×exp(exp(exp(x)))       
- exp(x), exp(exp(x)), exp(exp(exp(x)))      

In [130]:
my_func(0.2, 0.3)

-0.23493649049060805

In [252]:
reset_graph()
a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")
for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

In [253]:
z

<tf.Tensor 'add_199:0' shape=() dtype=float32>

In [254]:
grads

[<tf.Tensor 'gradients/AddN_99:0' shape=() dtype=float32>,
 <tf.Tensor 'gradients/AddN_100:0' shape=() dtype=float32>]

In [256]:
init

<tf.Operation 'init' type=NoOp>

- gradients() 함수는 (이 경우 mse)와 변수 목록 (이 경우에는 theta)을 취하고 각 변수와 관련하여 그래디언트를 계산하기 위해 변수 당 하나의 목록을 만듭니다. 
- 그래디언트 노드는 쎄타와 관련하여 MSE의 그래디언트 벡터를 계산합니다.
- 주로 그라데이션을 계산하는 4 가지 방법이 있습니다. 그것들은 표 9-2에 요약되어있다. 
- TensorFlow는 역 입력 자동 모드(reverse-mode autodiff)를 사용합니다. 이 모드는 입력이 많고 출력이 적을 때 완벽하고 (효율적이고 정확합니다), 종종 신경망 네트워크에서처럼 그러합니다. n outputs + 1 그래프 횡단에서 모든 입력에 대한 출력의 모든 부분 파생 값을 계산합니다.

<img src= './img/table_9_2.png'>

- appendix에 미분 방법의 4가지 방법이 나와있음. reverse-mode autodiff는 그래프를 순방향 (즉, 입력에서 출력까지)으로 진행하여 각 노드의 값을 계산합니다. 그런 다음 두 번째 패스를 수행하고, 이번에는 역방향으로 (즉, 출력에서 입력으로) 모든 부분 파생 값을 계산합니다. 

- reverse-mode autodiff은 특히 입력이 많고 출력이 적은 경우 매우 강력하고 정확한 기술입니다. 모든 입력에 대해 모든 출력의 모든 편미분도를 계산하기 위해 출력 당 하나의 순방향 통과와 역방향 통과가 필요하기 때문에. 가장 중요한 것은 임의 코드로 정의 된 함수를 처리 할 수 있다는 것입니다. 또한 미분 할 수 있는 함수를 처리 할 수 있습니다. 단, 미분 할 수 있는 점에서 부분 파생 값을 계산하도록 요청해야합니다.


### Using an optimizer
- 따라서 TensorFlow는 사용자의 그라디언트를 계산합니다. TensorFlow는 그래디언트 하강 옵티 마이저 (Gradient Descent Optiizer)를 포함하여 다양한 옵티마이저를 즉시 제공합니다. 위의 그래디언트 = ... 및 training_op = ... 행을 다음 코드로 대체하면 모든 것이 올바르게 작동합니다.

In [135]:
reset_graph()
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [133]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [134]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.714501
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.555572
Epoch 400 MSE = 0.548812
Epoch 500 MSE = 0.543636
Epoch 600 MSE = 0.539629
Epoch 700 MSE = 0.536509
Epoch 800 MSE = 0.534068
Epoch 900 MSE = 0.532147
Best theta:
[[ 2.06855249]
 [ 0.88740271]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145277]
 [-0.6375277 ]]


### Feeding data to the training algorithm

- 위의 코드를 수정하여 Mini-batch Gradient Descent를 구현해 보겠습니다. 이를 위해서는 모든 반복마다 X와 Y를 다음 미니 배치로 바꾸는 방법이 필요합니다. 가장 간단한 방법은 자리 표시 자 노드를 사용하는 것입니다. 이 노드는 실제로 계산을 수행하지 않기 때문에 특별합니다. 런타임시 출력하도록 지정한 데이터 만 출력합니다. 런타임에 자리 표시 자에 값을 지정하지 않으면 예외가 발생합니다.

- 자리 표시 자 노드를 만들려면 placeholder() 함수를 호출하고 출력 텐서의 데이터 형식을 지정해야합니다. 선택적으로 적용하려는 경우 모양을 지정할 수도 있습니다. 차원에 없음을 지정하면 "모든 크기"를 의미합니다. 예를 들어, 다음 코드는 자리 표시 자 노드 A와 노드 B = A + 5를 만듭니다. B를 평가할 때 A 값을 지정하는 eval () 메서드에 feed_dict를 전달합니다. A는 rank 2 (즉 2 차원이어야 함)이며 3 개의 열이 있어야합니다 (그렇지 않으면 예외가 발생 함). 그러나 행 수에는 제한이 없습니다.

In [257]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)

[[ 6.  7.  8.]]


In [264]:
type([[1, 2, 3]])

list

In [270]:
{A: [[1, 2, 3]]}

{<tf.Tensor 'Placeholder:0' shape=(?, 3) dtype=float32>: [[1, 2, 3]]}

In [271]:
{A: [[4, 5, 6], [7, 8, 9]]}

{<tf.Tensor 'Placeholder:0' shape=(?, 3) dtype=float32>: [[4, 5, 6],
  [7, 8, 9]]}

In [137]:
print(B_val_2)

[[  9.  10.  11.]
 [ 12.  13.  14.]]


- Mini-batch Gradient Descent를 구현하려면 기존 코드를 약간만 조정하면됩니다. 먼저 생성 단계에서 X와 Y의 정의를 변경하여 자리 표시 자 노드로 만듭니다.

In [272]:
reset_graph()
n_epochs = 1000
learning_rate = 0.01

In [273]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [274]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

- 그런 다음 배치 크기를 정의하고 총 배치 수를 계산하십시오.

In [149]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
n_batches

207

- 마지막으로 실행 단계에서 하나씩 미니 배치를 가져온 다음 둘 중 하나에 종속 된 노드를 평가할 때 feed_dict 매개 변수를 통해 X 및 y 값을 제공하십시오.

In [150]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [275]:
best_theta

array([[ 2.07033372],
       [ 0.86371452],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.80304712]], dtype=float32)

### Saving and restoring models
- 모델을 교육 한 후에는 매개 변수를 디스크에 저장해야 원하는 때 언제든지 다시 돌아와 다른 프로그램에서 사용하고 다른 모델과 비교하는 등의 작업을 수행 할 수 있습니다. 또한 트레이닝 중에 정기적 인 체크 포인트를 저장하기를 원할 것입니다. 따라서 트레이닝 중에 컴퓨터가 다운되는 경우 처음부터 다시 시작하지 않고 마지막 체크 포인트부터 계속할 수 있습니다.

- TensorFlow를 사용하면 모델을 쉽게 저장하고 복원 할 수 있습니다. 모든 변수 노드가 생성 된 후 생성 단계의 끝에서 Saver 노드를 생성 한 다음 실행 단계에서 모델을 저장하고 체크 포인트의 세션과 경로를 전달할 때마다 save () 메서드를 호출하기 만하면됩니다. 

- 모델을 복원하는 것 역시 쉽습니다. 위와 같이 생성 단계가 끝날 때 Saver를 만들었지 만 실행 단계가 시작될 때 init 노드를 사용하여 변수를 초기화하는 대신 restore () 메서드를 호출합니다 보호기 개체 :

In [276]:
n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

reset_graph()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.714501
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.555572
Epoch 400 MSE = 0.548812
Epoch 500 MSE = 0.543636
Epoch 600 MSE = 0.539629
Epoch 700 MSE = 0.536509
Epoch 800 MSE = 0.534068
Epoch 900 MSE = 0.532147


- 기본적으로 Saver는 모든 변수를 자체 이름으로 저장 및 복원하지만 더 많은 제어가 필요한 경우 저장 또는 복원 할 변수와 사용할 이름을 지정할 수 있습니다. 예를 들어 다음 Saver는 이름 가중치 아래에 theta 변수 만 저장하거나 복원합니다.

In [153]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [154]:
np.allclose(best_theta, best_theta_restored)

True

- "weights"와 같이 다른 이름으로 theta를로드하고 복원하는 보호기를 갖고 싶다면 다음을 수행하십시오.

In [155]:
saver = tf.train.Saver({"weights": theta})

- 기본적으로 세이버는 그래프 구조 자체를 확장자가 .meta 인 두 번째 파일에 저장합니다. tf.train.import_meta_graph () 함수를 사용하여 그래프 구조를 복원 할 수 있습니다. 이 함수는 그래프를 기본 그래프에로드하고 그래프 상태 (즉, 변수 값)를 복원하는 데 사용할 수있는 Saver를 반환합니다.

In [156]:
reset_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [157]:
np.allclose(best_theta, best_theta_restored)

True

- 즉, 그래프를 작성하는 데 필요한 Python 코드가 없어도 사전 모델을 가져올 수 있습니다. 이것은 모델을 조정하고 저장할 때 매우 편리합니다. 모델을 빌드 한 코드의 버전을 검색하지 않고도 이전에 저장된 모델을 로드 할 수 있습니다.

### Visualizing the graph and training curves using TensorBoard

- 이제 Mini-batch Gradient Descent를 사용하여 선형 회귀 모델을 학습하는 계산 그래프를 얻었으며 일정 간격으로 검사 점을 저장합니다. 그러나 우리는 여전히 print() 함수에 의존하여 훈련 중 진행 상황을 시각화합니다. 더 좋은 방법이 있습니다 : TensorBoard를 입력하십시오.

- 그래프의 정의를 제공 할 수 있으며이를 통해 탐색 할 수있는 훌륭한 인터페이스를 제공합니다. 이것은 그래프의 오류를 식별하고 병목 현상을 찾는 등 매우 유용합니다.

- TenorBoard에서 읽을 로그 디렉터리에 기록하는 것입니다. 프로그램을 실행할 때마다 다른 로그 디렉토리를 사용해야합니다. 

<img src ='./img/tfboard_1.png'>

<img src ='./img/tfboard_3.png'>

<img src ='./img/tfboard_5.png'>

In [277]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [278]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [279]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

- 첫 번째 줄은 MSE 값을 평가하고 요약이라는 TensorBoard 호환 바이너리 로그 문자열에 쓰는 노드를 그래프에 만듭니다. 
- 두 번째 줄은 요약 파일을 로그 디렉토리에 기록하는 데 사용하는 SummaryWriter를 만듭니다. 
- 첫 번째 매개 변수는 로그 디렉토리의 경로를 나타냅니다 (이 경우 현재 디렉토리와 관련하여 tf_logs / run-20160906091959 /와 같은 것). 두 번째 매개 변수 (선택 사항)는 시각화하려는 그래프입니다. 
- 생성시, SummaryWriter는 로그 디렉토리가 아직 존재하지 않으면 (그리고 필요한 경우 상위 디렉토리) 로그 파일을 작성하고 그래프 파일 정의를 이벤트 파일이라는 2진 로그 파일에 작성합니다.

In [172]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [173]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

- 다음으로 트레이닝 중에 mse_summary 노드를 정기적으로 평가하기 위해 실행 단계를 업데이트해야합니다 (예 : 10 개의 미니 배치마다). summary_writer를 사용하여 이벤트 파일에 쓸 수있는 요약을 출력합니다. 다음은 업데이트 된 코드입니다.

In [174]:
file_writer.close()

- 마지막으로, 프로그램 끝에서 SummaryWriter를 닫으려고합니다.

- 이제 이 프로그램을 실행하십시오. 로그 디렉터리를 만들고 이 디렉터리에 그래프 정의와 MSE 값을 모두 포함하는 이벤트 파일을 작성합니다. 

- 셸을 열고 작업 디렉토리로 이동 한 다음 ls -l tf_logs / run *을 입력하여 로그 디렉토리의 내용을 나열하십시오.

- 이제는 TensorBoard 서버를 가동 할 차례입니다. 가상 환경을 만든 경우 가상 환경을 활성화 한 다음 tensorboard 명령을 실행하여 서버를 시작하고 루트 로그 디렉토리를 가리 키십시오. 그러면 TensorBoard 웹 서버가 시작되어 포트 6006에서 수신 대기합니다 


```
ource env/bin/activate
tensorboard --logdir tf_logs/
Starting TensorBoard  on port 6006
(You can navigate to http://0.0.0.0:6006)
```

- 다음 브라우저를 열고 http : ///0.0.0.0:6006/ (또는 http : // localhost : 6006 /)로 이동하십시오. TensorBoard에 오신 것을 환영합니다! "Events"탭에서 오른쪽에 "MSE"가 표시되어야합니다. 이 버튼을 클릭하면 두 실행에 대한 훈련 중 MSE 플롯이 표시됩니다. 보고 싶은 실행을 선택 또는 선택 해제하거나, 확대 또는 축소하거나, 곡선 위에 마우스를 올리면 자세한 내용을 볼 수 있습니다.

In [164]:
best_theta

array([[ 2.07033372],
       [ 0.86371452],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.80304712]], dtype=float32)

### Name scopes

- 신경망과 같은보다 복잡한 모델을 다룰 때 그래프는 수천 개의 노드로 인해 쉽게 어수선하게 될 수 있습니다. 이를 피하기 위해 이름 범위를 만들어 관련 노드를 그룹화 할 수 있습니다. 예를 들어 위의 코드를 수정하여 "손실"이라는 이름 범위 내의 오류 및 mse 작업을 정의합니다.

In [281]:
reset_graph()

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

- 범위 내에 정의 된 각 op의 이름 앞에는 "loss /"가 붙습니다.

In [282]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [284]:
print(error.op.name)

loss/sub


In [285]:
print(mse.op.name)

loss/mse


- 범위 내에 정의 된 각 op의 이름 앞에는 "loss /"가 붙습니다.

In [189]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [190]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.07033372]
 [ 0.86371452]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.80304712]]


In [193]:
reset_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


### Modularity

- 두 개의 직각 선형 단위 (ReLU)의 출력을 더하는 그래프를 만들고 싶다고 가정 해보십시오. ReLU는 입력의 선형 함수를 계산하고 양의 값을 출력하거나 아니면 식 9-1과 같이 0을 출력합니다.

In [194]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

- 이러한 반복적인 코드는 유지하기 어렵고 오류가 발생하기 쉽습니다. 몇 가지 ReLUs를 추가하고 싶다면 더 악화 될 것입니다. 다행히 TensorFlow를 사용하면 DRY (스스로 반복하지 말 것)를 유지할 수 있습니다. 간단히 ReLU를 작성하는 함수를 작성하십시오. 다음 코드는 5 개의 ReLUs를 만들고 합계를 출력합니다.

In [195]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

- 노드를 만들 때 TensorFlow는 이름이 이미 있는지 확인한 다음 이름이 고유한지 확인하기 위해 밑줄과 밑줄을 추가합니다. 따라서 첫 번째 ReLU에는 "weights", "bias", "z"및 "relu"라는 노드 ( "MatMul"와 같은 기본 이름이있는 더 많은 노드)가 포함되어 있지만 두 번째 ReLU에는 "weights_1" "bias_1"등과 같이 세 번째 ReLU에는 "weights_2", "bias_2"등의 노드가 있습니다. TensorBoard는 이러한 시리즈를 식별하여 함께 정리하여 혼란을 줄입니다 

- 이름 범위를 사용하면 그래프를 훨씬 명확하게 만들 수 있습니다. relu() 함수의 모든 내용을 이름 범위 내로 이동하기 만하면됩니다. 그림 9-7은 결과 그래프를 보여줍니다. TensorFlow는 _1, _2 등을 추가하여 이름 범위의 이름을 고유하게 만드는 작업도 처리합니다.

In [196]:
file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

Exception ignored in: <generator object get_controller at 0x11cc1f780>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 3628, in get_controller
    if self.stack[-1] is not default:
IndexError: list index out of range


In [197]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    # not shown
        b = tf.Variable(0.0, name="bias")                             # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                      # not shown
        return tf.maximum(z, 0., name="max")                          # not shown

In [198]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

### Sharing Variables

- 그래프의 다양한 구성 요소간에 변수를 공유하려면 먼저 변수를 만든 다음 필요로하는 함수에 매개 변수로 전달하는 간단한 옵션이 있습니다. 예를 들어 모든 ReLUs에 대해 공유 임계 값 변수를 사용하여 ReLU 임계 값 (현재 0으로 하드 코딩 됨)을 제어하려고한다고 가정합니다. 먼저 변수를 생성 한 다음 relu() 함수에 전달하면됩니다.

In [199]:
reset_graph()

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

- 이것은 잘 작동합니다 : 이제 임계 값 변수를 사용하여 모든 ReLUs에 대한 임계 값을 제어 할 수 있습니다. 그러나 이처럼 많은 공유 매개 변수가있는 경우 항상 매개 변수로 매개 변수를 전달해야합니다. 

- 많은 사람들이 모델의 모든 변수를 포함하는 파이썬 사전을 작성하고 모든 함수에 전달합니다. 다른 것들은 각 모듈에 대한 클래스를 생성합니다 (예 : 클래스 매개 변수를 사용하여 공유 매개 변수를 처리하는 ReLU 클래스). 또 다른 옵션은 공유 변수를 첫 번째 호출시 relu () 함수의 속성으로 설정하는 것입니다.

In [200]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, relu.threshold, name="max")

- TensorFlow는 위의 솔루션보다 약간 깔끔하고 규범적인 코드로 이어질 수있는 또 다른 옵션을 제공합니다 4. 이 솔루션은 처음에는 이해하기가 약간 까다롭지 만 TensorFlow에서 많이 사용 되었기 때문에 약간의 세부 사항으로 들어가는 것이 좋습니다.

- 아이디어는 아직 존재하지 않으면 get_variable () 함수를 사용하여 공유 변수를 작성하거나 이미 존재하는 경우 재사용하는 것입니다. 원하는 동작 (생성 또는 재사용)은 현재 variable_scope ()의 특성에 의해 제어됩니다. 예를 들어, 다음 코드에서는 "relu / threshold"라는 이름의 변수를 만듭니다 (shape = ()이므로 초기 값으로 0.0을 사용).

In [201]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [202]:
reset_graph()

with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))

- 이전에 get_variable ()을 호출하여 변수가 이미 생성 된 경우이 코드는 예외를 발생시킵니다. 이렇게하면 실수로 변수를 재사용 할 수 없습니다. 변수를 재사용하려면 변수 범위의 재사용 속성을 True로 설정하여 명시 적으로 말해야합니다 (이 경우 모양이나 이니셜 라이저를 지정할 필요가 없습니다).

In [203]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

- 이 코드는 기존 "relu / threshold"변수를 가져 오거나 (존재하지 않거나 get_variable ()을 사용하여 생성되지 않은 경우) 예외를 발생시킵니다. 또는 범위의 reuse_vari ables () 메서드를 호출하여 블록 내에서 reuse 속성을 True로 설정할 수 있습니다.

In [204]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

- 이제 relu () 함수가 thres hold 변수에 매개 변수로 전달하지 않고 액세스 할 수있게하는 데 필요한 모든 요소가 있습니다.

In [205]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

In [206]:
file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

- 이 코드는 먼저 relu () 함수를 정의한 다음 relu / threshold 변수 (나중에 0.0으로 초기화되는 스칼라로)를 만들고 relu () 함수를 호출하여 5 ReLUs를 작성합니다. relu () 함수는 relu / threshold 변수를 재사용하고 다른 ReLU 노드를 만듭니다.

In [207]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("", default_name="") as scope:
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu8", tf.get_default_graph())
file_writer.close()

- get_variable ()을 사용하여 생성 된 변수의 이름은 항상 variable_scope의 이름을 접두어로 사용하여 지정되지만 (예 : "relu / thres hold") 모든 다른 노드 (tf.Variable ()로 만든 변수 포함)의 경우 variable scope는 새로운 이름 범위. 

- 특히 동일한 이름의 이름 범위가 이미 만들어진 경우 이름을 고유하게 만들기 위해 접미사가 추가됩니다. 예를 들어, 위의 코드에서 작성된 모든 노드 (임계 값 변수 제외)는 그림 9-8에 표시된 것처럼 "relu_1 /"에서 "relu_5 /"접두어로 접두사가 붙은 이름을 갖습니다.

- 임계 값 변수는 나머지 ReLU 코드가있는 relu () 함수 외부에서 정의되어야한다는 것은 다소 불행한 일입니다. 이를 해결하기 위해 다음 코드는 첫 번째 호출시 relu () 함수 내에서 임계 값 변수를 만든 다음 후속 호출에서 다시 사용합니다. 이제 relu () 함수는 이름 범위 또는 변수 공유에 대해 걱정할 필요가 없습니다. get_variable ()을 호출하기 만하면 임계 변수를 만들거나 재사용합니다 (어떤 경우인지 알 필요가 없습니다). 나머지 코드는 relu ()를 5 번 호출하고 첫 번째 호출에서는 reuse = False로 설정하고 다른 호출에서는 reuse = True로 설정해야합니다.

In [208]:
reset_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [209]:
file_writer = tf.summary.FileWriter("logs/relu9", tf.get_default_graph())
file_writer.close()

- 이것으로 TensorFlow에 대한 소개를 마칩니다. 다음 장, 특히 심 신경 네트워크, 길쌈 신경 네트워크, 반복 신경 네트워크 및 다중 스레드, 대기열, 다중 GPU 및 다중 서버를 사용하여 TensorFlow로 확장하는 방법과 관련된 여러 가지 작업을 수행하는 동안보다 고급 토픽에 대해 설명합니다. .